In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [3]:
df = pd.read_csv("covid_toy.csv")


In [4]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [5]:
df['cough'].unique()

array(['Mild', 'Strong'], dtype=object)

In [6]:
df.shape

(100, 6)

In [7]:
df.isnull().sum()

,0
age,0
gender,0
fever,10
cough,0
city,0
has_covid,0


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'], test_size = 0.2)

In [9]:
X_train

,age,gender,fever,cough,city
56,71,Male,NaN,Strong,Kolkata
90,59,Female,99.0,Strong,Delhi
67,65,Male,99.0,Mild,Bangalore
54,60,Female,99.0,Mild,Mumbai
95,12,Female,104.0,Mild,Bangalore
...,...,...,...,...,...
76,80,Male,100.0,Mild,Bangalore
21,73,Male,98.0,Mild,Bangalore
32,34,Female,101.0,Strong,Delhi
40,49,Female,102.0,Mild,Delhi


In [10]:
# adding simple imputer to fever col
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[["fever"]])
# now for the test data
X_test_fever = si.fit_transform(X_test[["fever"]])

X_train_fever.shape

(80, 1)

In [11]:
X_train_fever

array([[100.64285714],
       [ 99.        ],
       [ 99.        ],
       [ 99.        ],
       [104.        ],
       [101.        ],
       [103.        ],
       [104.        ],
       [ 98.        ],
       [ 98.        ],
       [100.        ],
       [103.        ],
       [100.        ],
       [100.64285714],
       [ 98.        ],
       [103.        ],
       [ 98.        ],
       [104.        ],
       [104.        ],
       [ 98.        ],
       [101.        ],
       [101.        ],
       [101.        ],
       [100.64285714],
       [101.        ],
       [100.64285714],
       [102.        ],
       [102.        ],
       [100.64285714],
       [ 98.        ],
       [100.64285714],
       [103.        ],
       [ 99.        ],
       [101.        ],
       [100.        ],
       [101.        ],
       [102.        ],
       [ 98.        ],
       [ 98.        ],
       [102.        ],
       [101.        ],
       [102.        ],
       [100.        ],
       [100

In [12]:
# ordinal encoding ->
oe = OrdinalEncoder(categories=[['Mild', 'Strong']])
X_train_cough = oe.fit_transform(X_train[["cough"]])
# test data
X_test_cough = oe.fit_transform(X_test[["cough"]])

In [27]:
# Onehot Encoding -> gender, city
ohe = OneHotEncoder(drop='first', sparse_output=False)
X_train_gender_city = ohe.fit_transform(X_train[["gender", "city"]])
# test data
X_test_gender_city = ohe.fit_transform(X_test[["gender", "city"]])

In [22]:
X_train_gender_city.shape

(80, 4)

In [23]:
# Now I have to concatinate all the columns for that I have to extarct the age from the main column
X_train_age = X_train.drop(columns=['gender', 'fever', 'cough', 'city']).values
# test data
X_test_age = X_test.drop(columns=['gender', 'fever', 'cough', 'city']).values

In [28]:
X_train_tranformed  = np.concatenate((X_train_age, X_train_fever, X_train_gender_city, X_train_cough), axis=1)
# test data
X_test_tranformed  = np.concatenate((X_test_age, X_test_fever, X_test_gender_city, X_test_cough), axis=1)

In [31]:
from re import X
X_train_tranformed.shape, X_test_tranformed.shape

((80, 7), (20, 7))

In [32]:
### Column Tranformer !!!

In [33]:
from sklearn.compose import ColumnTransformer

In [37]:
transformer = ColumnTransformer(transformers=[ ('tnf1', SimpleImputer(), ['fever'])
                                              ,('tnf2', OrdinalEncoder(categories=[['Mild', 'Strong']]), ['cough'])
                                              ,('tnf3', OneHotEncoder(sparse_output=False, drop='first'), ['gender', 'city'])



                                              ],remainder='passthrough')
# remainder leaves the coulumn as it is
# If we put remainder =  drop , the it will drop the columns

In [39]:
transformer.fit_transform(X_train)

array([[100.64285714,   1.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  71.        ],
       [ 99.        ,   1.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  59.        ],
       [ 99.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  65.        ],
       [ 99.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  60.        ],
       [104.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  12.        ],
       [101.        ,   1.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  68.        ],
       [103.        ,   1.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  46.        ],
       [104.        ,   1.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  34.        ],
       [ 98.        ,   1.        ,   0.        ,   0.        ,
          0.    